# Exploratory Data Analysis: Birth Control Mental Health Posts

## Learning Objectives
- Understand the structure of collected Reddit data
- Perform basic statistical analysis
- Visualize text data distributions
- Identify patterns and prepare for NLP/LLM tasks

## What is EDA?
Exploratory Data Analysis (EDA) is the critical first step in any ML/AI project:
- **Understand your data**: What do you actually have?
- **Identify quality issues**: Missing data, duplicates, errors
- **Find patterns**: What insights emerge before modeling?
- **Guide decisions**: What preprocessing/modeling approach makes sense?

Rule of thumb: Spend 80% of time on data quality, 20% on modeling.

In [ ]:
# Import required libraries
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime
from collections import Counter
import re

# Visualization settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Display settings for pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

## 1. Load and Inspect Data

First, let's load our collected Reddit posts and examine the structure.

In [ ]:
# Find the most recent data file
data_dir = Path('../data/raw')
data_files = list(data_dir.glob('reddit_mental_health_posts_*.json'))

if not data_files:
    print("⚠️  No data files found!")
    print("   Please run: python src/data_collection/reddit_collector.py")
else:
    # Use the most recent file
    latest_file = max(data_files, key=lambda p: p.stat().st_mtime)
    print(f"📂 Loading: {latest_file.name}")
    
    # Load JSON data
    with open(latest_file, 'r', encoding='utf-8') as f:
        posts_raw = json.load(f)
    
    # Convert to pandas DataFrame for easier analysis
    # LEARNING: DataFrames are the standard for tabular data in Python
    df = pd.DataFrame(posts_raw)
    
    print(f"✓ Loaded {len(df)} posts")
    print(f"\nDataFrame shape: {df.shape[0]} rows × {df.shape[1]} columns")

In [ ]:
# Examine first few rows
# LEARNING: Always inspect raw data before analysis
df.head()

In [ ]:
# Data types and memory usage
df.info()

In [ ]:
# Statistical summary
df.describe()

## 2. Data Quality Checks

Before any analysis, we need to check data quality:
- Missing values
- Duplicates
- Data consistency

In [ ]:
# Check for missing values
print("Missing Values:")
print("=" * 40)
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Percentage': missing_pct
})
print(missing_df[missing_df['Missing Count'] > 0])

# INSIGHT: Empty selftext is common for link posts
# We'll filter for text posts later

In [ ]:
# Check for duplicates
duplicates = df.duplicated(subset=['id']).sum()
print(f"\nDuplicate posts: {duplicates}")

if duplicates > 0:
    print("Removing duplicates...")
    df = df.drop_duplicates(subset=['id'], keep='first')
    print(f"✓ After deduplication: {len(df)} posts")

In [ ]:
# Filter for text posts only (where we have content to analyze)
text_posts = df[df['is_self'] == True].copy()
text_posts = text_posts[text_posts['text_length'] > 0]

print(f"Text posts with content: {len(text_posts)} / {len(df)}")
print(f"Percentage: {len(text_posts)/len(df)*100:.1f}%")

## 3. Distribution Analysis

Understanding the distribution of our data helps us:
- Identify outliers
- Understand typical post characteristics
- Plan preprocessing strategies

In [ ]:
# Subreddit distribution
subreddit_counts = text_posts['subreddit'].value_counts()

fig, ax = plt.subplots(figsize=(10, 6))
subreddit_counts.plot(kind='bar', ax=ax)
ax.set_title('Posts by Subreddit', fontsize=14, fontweight='bold')
ax.set_xlabel('Subreddit')
ax.set_ylabel('Number of Posts')
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.show()

print("\nPosts per subreddit:")
print(subreddit_counts)

In [ ]:
# Text length distribution
# LEARNING: Text length affects:
# - LLM token usage (and cost!)
# - Processing time
# - Information density

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram
axes[0].hist(text_posts['text_length'], bins=50, edgecolor='black')
axes[0].set_title('Distribution of Post Lengths', fontweight='bold')
axes[0].set_xlabel('Text Length (characters)')
axes[0].set_ylabel('Frequency')
axes[0].axvline(text_posts['text_length'].median(), color='red', 
                linestyle='--', label=f'Median: {text_posts["text_length"].median():.0f}')
axes[0].legend()

# Box plot
axes[1].boxplot(text_posts['text_length'])
axes[1].set_title('Post Length Box Plot', fontweight='bold')
axes[1].set_ylabel('Text Length (characters)')

plt.tight_layout()
plt.show()

print("\nText Length Statistics:")
print(text_posts['text_length'].describe())

In [ ]:
# Post score (engagement) distribution
fig, ax = plt.subplots(figsize=(12, 6))
ax.hist(text_posts['score'], bins=50, edgecolor='black')
ax.set_title('Distribution of Post Scores (Engagement)', fontweight='bold')
ax.set_xlabel('Score (upvotes - downvotes)')
ax.set_ylabel('Frequency')
ax.axvline(text_posts['score'].median(), color='red', 
           linestyle='--', label=f'Median: {text_posts["score"].median():.0f}')
ax.legend()
plt.tight_layout()
plt.show()

print("\nScore Statistics:")
print(text_posts['score'].describe())

## 4. Temporal Analysis

When were these posts made? Are there patterns over time?

In [ ]:
# Convert timestamp to datetime
text_posts['created_datetime'] = pd.to_datetime(text_posts['created_date'])
text_posts['year_month'] = text_posts['created_datetime'].dt.to_period('M')

# Posts over time
temporal_dist = text_posts.groupby('year_month').size()

fig, ax = plt.subplots(figsize=(14, 6))
temporal_dist.plot(kind='line', marker='o', ax=ax)
ax.set_title('Posts Over Time', fontsize=14, fontweight='bold')
ax.set_xlabel('Month')
ax.set_ylabel('Number of Posts')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Text Content Analysis

Now let's analyze the actual text content - this is where NLP begins!

In [ ]:
# Combine title and text for full content analysis
text_posts['full_text'] = text_posts['title'] + ' ' + text_posts['selftext']

# Most common words (simple tokenization)
# LEARNING: This is basic NLP - we'll use advanced methods later
def simple_tokenize(text):
    """Basic word tokenization (lowercase, split on spaces)"""
    if pd.isna(text):
        return []
    # Convert to lowercase and split
    words = re.findall(r'\b\w+\b', text.lower())
    return words

# Collect all words
all_words = []
for text in text_posts['full_text']:
    all_words.extend(simple_tokenize(text))

# Count frequencies
word_freq = Counter(all_words)

# Remove common English stop words (basic list)
stop_words = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 
              'of', 'with', 'by', 'from', 'as', 'is', 'was', 'are', 'been', 'be',
              'have', 'has', 'had', 'do', 'does', 'did', 'will', 'would', 'could',
              'should', 'may', 'might', 'must', 'can', 'this', 'that', 'these', 'those',
              'i', 'you', 'he', 'she', 'it', 'we', 'they', 'my', 'your', 'his', 'her',
              'its', 'our', 'their', 'me', 'him', 'us', 'them', 'im', 've', 't', 's'}

filtered_freq = {word: count for word, count in word_freq.items() 
                 if word not in stop_words and len(word) > 2}

# Top 30 most common words
top_words = Counter(filtered_freq).most_common(30)

# Visualize
words, counts = zip(*top_words)
fig, ax = plt.subplots(figsize=(12, 8))
ax.barh(words, counts)
ax.set_title('Top 30 Most Common Words (excluding stop words)', fontweight='bold')
ax.set_xlabel('Frequency')
ax.invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# Mental health keyword frequency
# LEARNING: Domain-specific keyword analysis
mental_health_terms = {
    'depression': ['depression', 'depressed', 'depressing'],
    'anxiety': ['anxiety', 'anxious', 'panic'],
    'mood': ['mood', 'moods', 'emotional', 'emotions'],
    'anger': ['anger', 'angry', 'rage', 'irritable', 'irritability'],
    'crying': ['crying', 'cry', 'cried', 'tears'],
    'suicidal': ['suicidal', 'suicide', 'kill myself'],
    'mental_health': ['mental health', 'mentally']
}

def count_term_occurrences(text, terms):
    """Count how many times any variant of a term appears"""
    if pd.isna(text):
        return 0
    text_lower = text.lower()
    return sum(text_lower.count(term) for term in terms)

# Count occurrences for each category
term_counts = {}
for category, terms in mental_health_terms.items():
    term_counts[category] = text_posts['full_text'].apply(
        lambda x: count_term_occurrences(x, terms)
    ).sum()

# Visualize
fig, ax = plt.subplots(figsize=(10, 6))
categories = list(term_counts.keys())
counts = list(term_counts.values())
ax.bar(categories, counts)
ax.set_title('Mental Health Term Frequencies', fontweight='bold')
ax.set_xlabel('Category')
ax.set_ylabel('Total Occurrences')
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.show()

print("\nMental Health Term Counts:")
for category, count in sorted(term_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"  {category}: {count}")

## 6. Sample Posts Review

Let's look at actual examples to understand the data qualitatively.

In [ ]:
# Show a few example posts
print("Sample Posts (high engagement):")
print("=" * 80)

# Get top posts by score
top_posts = text_posts.nlargest(3, 'score')

for idx, post in top_posts.iterrows():
    print(f"\n[{post['subreddit']}] Score: {post['score']}")
    print(f"Title: {post['title']}")
    print(f"Text: {post['selftext'][:300]}...")
    print("-" * 80)

## 7. Summary and Next Steps

### What We Learned:
1. **Data Quality**: Checked for missing values, duplicates
2. **Distributions**: Understood text length, engagement patterns
3. **Content**: Identified key mental health terms
4. **Temporal**: Saw when posts were created

### Next Steps for Your Learning Journey:
1. **Text Preprocessing**: Clean and normalize text
2. **Named Entity Recognition (NER)**: Extract drug names, symptoms
3. **LLM-based Extraction**: Use Claude/GPT to extract structured data
4. **Sentiment Analysis**: Analyze emotional tone
5. **Knowledge Graph**: Build relationships between entities

### Data Quality Assessment:
- Are there enough posts for analysis?
- Is the data representative?
- Do we need more data from specific subreddits?
- What preprocessing is needed?

In [ ]:
# Export summary statistics
summary = {
    'total_posts': len(df),
    'text_posts': len(text_posts),
    'subreddits': text_posts['subreddit'].value_counts().to_dict(),
    'avg_text_length': text_posts['text_length'].mean(),
    'avg_score': text_posts['score'].mean(),
    'date_range': {
        'earliest': text_posts['created_datetime'].min().isoformat(),
        'latest': text_posts['created_datetime'].max().isoformat()
    },
    'mental_health_terms': term_counts
}

# Save summary
output_path = Path('../outputs/reports/eda_summary.json')
output_path.parent.mkdir(parents=True, exist_ok=True)

with open(output_path, 'w') as f:
    json.dump(summary, f, indent=2)

print(f"\n✓ Summary saved to: {output_path}")